# Generating the Beam Example System
This notebook generates the beam models that will be used in many of the FBS examples that are used to demonstrate the common issues. A graphic of the different beam subcomponents, referred to as beam one and beam two, are shown in the images below. Beam one is the blue beam and beam two is the red beam. Both these beams are 50 cm long, more details on beams one and two will be provided below.

```{note}
Beams one and two are mirrored copies of each other.
```

```{note}
These beams have both a torsion and translating spring attached at one end, as shown in the images. These springs are added to avoid issues with singular stiffness matrices
```

![Beam1](./Images/beam1.svg)

![Beam2](./Images/beam2.svg)

The coupled beam system is generated by overlapping beams one and two by 10 cm and and attaching them to one-another at three (evenly spaced) points at each end of the beam, as shown as the green dots in the image below. More details on the coupling will be provided below. 

```{note}
The beams are directly on top of each other in the model. They are shown offset from each other in the graphic for visualization purposes only. 
```

![CoupledBeams](./Images/CoupledBeams.svg)

The data in this notebook will be written to SDynPy System and TransferFunctionArray objects to simplify the bookkeeping for various computations. These SDynPy objects will be saved to the root folder of the book and will be re-loaded into the notebooks for the different examples, as necessary. 

In [1]:
import sdynpy as sdpy
import numpy as np

(sec:beam_generation)=
## Creating the Beam Mass and Stiffness Matrices
As previously mentioned, beams one and two are exactly the same, except for the location of the torsion and translating springs. The mass and stiffness matrices for the beams are generated with Euler-Bernouli beam theory using a standard function in SDynPy. For brevity in the examples, the beams are being modeled in two dimensions only (in plane with the screen), without any axial stiffness elements. As such, the modeled beams only include shear and torsion motion. The specific properties for the beams are listed in the code block below.

```{note}
The beam material properties are for aluminum and the number of nodes was selected to give a 2.5mm spacing between nodes (for use in some of the later examples). The beam dimensions were selected to get a reasonable modal density to ~5,000 Hz.
```

In [ ]:
beam_stiffness, beam_mass = sdpy.beam.beamkm_2d(length = 0.5, # m
                                                width = 0.05, # m 
                                                height = 0.01, # m
                                                nnodes = 201, # 201 nodes, 2.5 mm spacing 
                                                E = 69.8e9, # youngs modulus, pa
                                                rho = 2700, # density, kg/m^3
                                                nu = 0.33, # Poisson's ratio 
                                                axial=False)

### Making the Beam One System Object
The beam one nodes are labeled as 1001-1201 (from the left to right side). 

In [ ]:
# SDynPy objects require a DOF list for the nodes in the models of the systems
beam1_dof = sdpy.coordinate_array(node=np.arange(201)+1001, direction=[3,5], 
                                  force_broadcast=True)

# Defining the SDynPy system to simplify the computations
beam1_system = sdpy.System(beam1_dof, beam_mass, beam_stiffness)

As previously mentioned, the beam is connected to ground via a translating and torsion spring at its left end, which must be manually added to the stiffness matrix. The spring stiffnesses were selected to be much softer than the element stiffnesses of the beam, while still effecting the rigid body modes of the beam. 

In [4]:
# Translating spring attachment to ground
beam1_system.stiffness[0,0] += 1e4

# Torsion spring attachment to ground
beam1_system.stiffness[1,1] += 1e2

The damping matrix is added to the system using a standard function in SDynPy to have a damping ratio of 0.5% for each mode.

In [5]:
beam1_system.assign_modal_damping(0.005) # 0.5% damping

#### Beam One Modal Properties and FRFs
The natural frequencies for the first nine modes of beam one are listed below, where the first two natural frequencies are rigid body modes and the remaining are the typical bending modes. 

In [6]:
print(beam1_system.eigensolution(num_modes=9))

   Index,  Frequency,    Damping,     # DoFs
    (0,),     6.3025,    0.5003%,        402
    (1,),    39.1812,    0.5000%,        402
    (2,),   215.6093,    0.5000%,        402
    (3,),   580.7172,    0.5000%,        402
    (4,),  1133.5516,    0.5000%,        402
    (5,),  1871.0716,    0.5000%,        402
    (6,),  2793.1680,    0.5000%,        402
    (7,),  3899.7624,    0.5000%,        402
    (8,),  5190.8283,    0.5000%,        402



The FRFs are created using a concatenation of three SDynPy `CoordinateArrays`, which are both the response and reference DOFs for the FRFs. The intention for the different `CoordinateArrays` is:

1. `beam1_coupling_dofs` - These are the intended DOFs where beam one is attached to beam two. These nodes are selected based on knowledge of the node locations on the beam.
2. `beam1_incorrect_coupling_dofs` - These are the DOFs that will be used in an example where there is a positional error between the coupling DOFs that are used in the substructure coupling and the actual coupling. Note that these locations are with +/- 5 mm of the true coupling DOF.
3. `beam1_evaluation_dofs` - These are the reference and response DOFs on beam one that will be used when evaluating the accuracy of the substructure coupling in all the examples. They are chosen to be evenly spaced from the left to right side of the beam, without going into the portion of the beam that is overlapped with beam two. 

In [ ]:
beam1_coupling_dofs = sdpy.coordinate_array(node=[1161, 1181, 1201], direction=[3,5], 
                                            force_broadcast=True)
beam1_incorrect_coupling_dofs = sdpy.coordinate_array(node=[1160, 1183, 1200], direction=[3,5], 
                                                      force_broadcast=True)
beam1_evaluation_dofs = sdpy.coordinate_array(node=[1001, 1051, 1101, 1151], direction=3)

beam1_frf_dofs = np.concatenate((beam1_coupling_dofs, beam1_incorrect_coupling_dofs, beam1_evaluation_dofs))

The FRFs are being computed from the modes of the system for computational efficiency. The FRFs are initially computed to 100,000 Hz, which is much higher frequency than will be evaluated in the substructure coupling and is being used to mitigate modal truncation issues. After the FRFs are computed, the frequency axis is trimmed to 0-5,000 Hz using the `extract_elements_by_abscissa` method. 

```{note}
The `displacement_derivative` kwarg is set to two so the resulting FRFs are in accelerance format.
```

In [ ]:
beam1_modes = beam1_system.eigensolution()

beam1_frfs = beam1_modes.compute_frf(np.arange(100001), beam1_frf_dofs, beam1_frf_dofs, 
                                     displacement_derivative=2)
beam1_frfs = beam1_frfs.extract_elements_by_abscissa(0, 5000)

### Making the Beam Two System Object
The beam two nodes are labeled 2001-2201 (from left to right). The beam two system and FRFs are created with the exact same processes as beam one, except for the springs to ground being connected to the right end of the beam. Similarly, the `CoordinateArrays` for beam two use the same logic as the `CoordinateArrays` for beam one.

In [ ]:
# SDynPy objects require a DOF list for the nodes in the models of the systems
beam2_dof = sdpy.coordinate_array(node=np.arange(201)+2001, direction=[3,5], 
                                  force_broadcast=True)

# Defining the SDynPy system to simplify the computations
beam2_system = sdpy.System(beam2_dof, beam_mass, beam_stiffness)

In [10]:
beam2_system.stiffness[-2,-2] += 1e4
beam2_system.stiffness[-1,-1] += 1e2

In [11]:
beam2_system.assign_modal_damping(0.005) # 0.5% damping

#### Beam Two Modal Properties and FRFs
The natural frequencies are the exact same for beams one and two since the mass and stiffness matrices did not change. The mode shapes are also the same, except for the flip about the Z-axis based on the change in the ground spring attachment location. 

In [12]:
print(beam2_system.eigensolution(num_modes=9))

   Index,  Frequency,    Damping,     # DoFs
    (0,),     6.3030,    0.5003%,        402
    (1,),    39.1817,    0.5000%,        402
    (2,),   215.6093,    0.5000%,        402
    (3,),   580.7173,    0.5000%,        402
    (4,),  1133.5516,    0.5000%,        402
    (5,),  1871.0716,    0.5000%,        402
    (6,),  2793.1680,    0.5000%,        402
    (7,),  3899.7624,    0.5000%,        402
    (8,),  5190.8283,    0.5000%,        402



```{note}
The `beam2_incorrect_coupling_dofs` have different location errors (relative to the true coupling DOF) compared to `beam1_incorrect_coupling_dofs`. All the incorrect coupling DOF locations are still within +/-5 mm of the true coupling DOF.
```

In [ ]:
beam2_coupling_dofs = sdpy.coordinate_array(node=[2001, 2021, 2041], direction=[3,5], 
                                            force_broadcast=True)
beam2_incorrect_coupling_dofs = sdpy.coordinate_array(node=[2002, 2020, 2043], direction=[3,5], 
                                                      force_broadcast=True)
beam2_evaluation_dofs = sdpy.coordinate_array(node=[2051, 2101, 2151, 2201], direction=3)

beam2_frf_dofs = np.concatenate((beam2_coupling_dofs, beam2_incorrect_coupling_dofs, 
                                 beam2_evaluation_dofs))

In [ ]:
beam2_modes = beam2_system.eigensolution()

beam2_frfs = beam2_modes.compute_frf(np.arange(100001), beam2_frf_dofs, beam2_frf_dofs, 
                                     displacement_derivative=2)
beam2_frfs = beam2_frfs.extract_elements_by_abscissa(0, 5000)

## Creating the Coupled System
The coupled system is created via substructuring in the physical domain with standard SDynPy functions. This is done by first creating the `combined_system`, which is a SDynPy `System` object with system matrices from both beams one and two in block diagonal format without the coupling constraints. The `System` object with the coupling constraint is made with the `substructure_by_coordinate` method for the system object and it is labeled `coupled_system`. 

As previously mentioned, there is 10 cm of overlap between beams with three evenly spaced coupling nodes. There is one coupling node at the end of each beam, where the accompanying coupling node on the second beam is at the overlapping position.

In [15]:
combined_system = sdpy.System.concatenate([beam1_system, beam2_system])

In [ ]:
# Need to make an nx2 array of coupling DOFs for the substructuring function
coupling_coordinate_pairs = np.column_stack((beam1_coupling_dofs, beam2_coupling_dofs))

# Doing the substructuring in the physical domain using the combined system and coordinate pairs
coupled_system = combined_system.substructure_by_coordinate(coupling_coordinate_pairs)

### Coupled System Modal Properties and FRFs
The FRFs for the coupled system are created using the same procedure as the individual beams. Note that there are many more natural frequencies for the coupled system in the band of interest compared to the individual beams. 

In [46]:
print(coupled_system.eigensolution(num_modes=15))

   Index,  Frequency,    Damping,     # DoFs
    (0,),    16.4120,    0.3210%,        804
    (1,),    34.7524,    0.4592%,        804
    (2,),    79.8826,    0.3209%,        804
    (3,),   185.4527,    0.4337%,        804
    (4,),   351.6890,    0.4139%,        804
    (5,),   577.0466,    0.4800%,        804
    (6,),   869.5430,    0.4489%,        804
    (7,),  1194.9697,    0.4848%,        804
    (8,),  1619.2261,    0.4694%,        804
    (9,),  2041.4236,    0.4816%,        804
   (10,),  2593.6483,    0.4840%,        804
   (11,),  3122.7647,    0.4789%,        804
   (12,),  3785.1733,    0.4933%,        804
   (13,),  4442.4619,    0.4792%,        804
   (14,),  5191.6887,    0.4967%,        804



In [47]:
coupled_system_frf_dofs = np.concatenate((beam1_frf_dofs, beam2_frf_dofs))

In [ ]:
coupled_system_modes = coupled_system.eigensolution()

coupled_system_frfs = coupled_system_modes.compute_frf(np.arange(100001), 
                                                       coupled_system_frf_dofs, 
                                                       coupled_system_frf_dofs, 
                                                       displacement_derivative=2)
coupled_system_frfs = coupled_system_frfs.extract_elements_by_abscissa(0, 5000)

## Saving the Data for Future Use
This section saves the SDynPy System and TransferFunctionArray objects to the root folder of the book for future use by other notebooks for the different examples. 

In [49]:
beam1_system.save('./example_systems/beam1_system.npz')
beam1_frfs.save('./example_frfs/beam1_frfs.npz')

beam2_system.save('./example_systems/beam2_system.npz')
beam2_frfs.save('./example_frfs/beam2_frfs.npz')

coupled_system.save('./example_systems/coupled_beam_system.npz')
coupled_system_frfs.save('./example_frfs/coupled_beam_system_frfs.npz')